# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])


[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
twits_data = twits['data']
print(len(twits_data))



1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

print(messages[:10])

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', '$AMD reveal yourself!', '$AAPL Why the drop? I warren Buffet taking out his position?', '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', '$BAC ok good we&#39;re not dropping in price over the weekend, lol', '$AMAT - Daily Chart, we need to get back to above 50.', '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... if BO, then bingo... what is the odds?']


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

# Added a test cell below to ensure i have the correct regex

In [5]:
teststring = " this is a @whatever $whatever $WHAT http://www.cnn.com, test #&$% &#$ &5 #&@#@ 7718123 &#@ &"
substring = r'\@[a-zA-Z0-9]+|\$[a-zA-Z0-9]+|http[a-zA-Z0-9:.\/]+'
notalpha = r'\W'
newstring = re.sub(substring, ' ', teststring)
print(newstring)
secondclean = re.sub(notalpha, ' ', newstring)
print(secondclean)

 this is a        , test #&$% &#$ &5 #&@#@ 7718123 &#@ &
 this is a          test           5       7718123      


### Pre-Processing

In [6]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement
    # replace
    substring = r'\@[a-zA-Z0-9]+|\$[a-zA-Z0-9]+|http[a-zA-Z0-9:.\/]+'
    messages_cleaned = re.sub(substring,' ',message)
    
    # Lowercase the twit message
    text = messages_cleaned.lower()
    #print("first cleaning", text[:20])
    
    # Replace URLs with a space in the message
    #text = pass - already did with multiple substring above
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    #text = pass - already did with multiple substring above
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    #text = pass - already did with multiple substring above

    # Replace everything not a letter with a space
    
    text = re.sub(r'\W',' ',text)
    #print("second cleaning", text[:20])
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split(' ')

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(token) for token in tokens if len(token)>1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [7]:
# TODO Implement

tokenized = [preprocess(twit) for twit in messages]
print(tokenized[0:20])

[['great', 'buy', 'at', '26', '00', 'ill', 'wait'], [], ['staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', '350', '00', 'our', 'own', 'verdict', 'is', 'buy'], ['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits'], ['reveal', 'yourself'], ['why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position'], ['bear', 'have', 'reason', 'on', '06', '29', 'to', 'pay', 'more', 'attention', 'ba'], ['ok', 'good', 'we', '39', 're', 'not', 'dropping', 'in', 'price', 'over', 'the', 'weekend', 'lol'], ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above', '50'], ['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'in', 'month', 'back', 'to', '12', 'if', 'bo', 'then', 'bingo', 'what', 'is', 'the', 'odds'], ['strong', 'buy'], ['short', 'ratio', 'is', '17', 'at', '2018', '06', '15', 'and', 'short', 'to', 'float', 'is', '42', 

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [8]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
token_text = [item for sublist in tokenized for item in sublist if sublist!=""]

mydict = Counter(token_text)

bow = dict(mydict)
print(bow)

{'great': 17759, 'buy': 79734, 'at': 140654, '26': 15507, '00': 23802, 'ill': 891, 'wait': 11387, 'staanalystalert': 4072, 'for': 282545, 'jefferies': 1171, 'maintains': 965, 'with': 71949, 'rating': 16762, 'of': 218778, 'hold': 22832, 'setting': 3920, 'target': 18395, 'price': 40109, 'usd': 2952, '350': 1978, 'our': 12346, 'own': 9137, 'verdict': 3745, 'is': 288698, 'heard': 1364, 'there': 26037, 'guy': 13457, 'who': 17995, 'know': 20342, 'someone': 5195, 'think': 24420, 'somebody': 1100, 'something': 6047, 'on': 247481, 'stocktwits': 101328, 'reveal': 253, 'yourself': 1210, 'why': 22143, 'the': 406247, 'drop': 18235, 'warren': 1174, 'buffet': 1403, 'taking': 6536, 'out': 57439, 'his': 6363, 'position': 11953, 'bear': 33804, 'have': 53413, 'reason': 8055, '06': 8091, '29': 9648, 'to': 396084, 'pay': 7866, 'more': 56847, 'attention': 3559, 'ba': 1171, 'ok': 4576, 'good': 44604, 'we': 65066, '39': 198928, 're': 13278, 'not': 68961, 'dropping': 2296, 'in': 212549, 'over': 48047, 'weekend

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [9]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
totals = sum(bow.values())
freqs = {key:(value/totals) for key,value in bow.items()}
print(bow['the'])
print(freqs['the'])

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.0000008

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 0.008

print(len(freqs))
# The k most common words in the corpus. Use `high_cutoff` as the k.
#k = {key for (key, freq) in freqs if freqs[key] > high_cutoff}
most_common = []
least_common = []
least_common_amount = []
for word in freqs:
    if freqs[word]> high_cutoff:
        most_common.append(word)
    if freqs[word]<low_cutoff:
        least_common.append(word)
        least_common_amount.append(freqs[word])
    
#K_most_common = sorted(freqs.items())
K_most_common = most_common


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common, len(K_most_common))
print("20 least common", least_common[:20], ", with total number of ", len(least_common))
print("and freqency of ", least_common_amount[:20])
len(filtered_words) 

406247
0.020776200490971453
184568
['for', 'of', 'is', 'on', 'the', 'to', '39', 'in', 'and', 'this', 'it', 'amp'] 12
20 least common ['raisinf', '18140', 'froend', 'andromeda', 'windowsphone', 'windows10', '1d21bec34e96', 'firearm', 'salman', 'dil', '5ema', 'saber', 'denominator', 'ricciardi', 'tradenet', 'campaign_id', '7010o000001pnwd', 'affiliateorg', 'affiliateid', '36619'] , with total number of  163895
and freqency of  [5.114179425564116e-08, 5.114179425564116e-08, 5.114179425564116e-08, 3.579925597894881e-07, 5.114179425564116e-08, 1.5342538276692348e-07, 5.114179425564116e-08, 1.5342538276692348e-07, 4.0913435404512926e-07, 1.5342538276692348e-07, 4.0913435404512926e-07, 6.64843325323335e-07, 6.64843325323335e-07, 2.0456717702256463e-07, 2.557089712782058e-07, 5.114179425564116e-08, 5.114179425564116e-08, 5.114179425564116e-08, 5.114179425564116e-08, 5.114179425564116e-08]


20661

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [10]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
#vocab = {word: bow[word] for word in bow if word in filtered_words}
vocab ={word: index for index, word in enumerate(filtered_words, 1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {count: word for word, count in vocab.items()}
# tokenized with the words not in `filtered_words` removed.            
filtered = [[word for word in twit if word in vocab] for twit in tokenized]

    

In [11]:
# further testing
counter = 0
limit = 20
for id in id2vocab:
    print (id, "->", id2vocab[id])
    counter+=1
    if counter == limit:
        break
counter = 0
limit = 20
for word in vocab:
    print (word, "->", vocab[word])
    counter+=1
    if counter == limit:
        break

1 -> great
2 -> buy
3 -> at
4 -> 26
5 -> 00
6 -> ill
7 -> wait
8 -> staanalystalert
9 -> jefferies
10 -> maintains
11 -> with
12 -> rating
13 -> hold
14 -> setting
15 -> target
16 -> price
17 -> usd
18 -> 350
19 -> our
20 -> own
great -> 1
buy -> 2
at -> 3
26 -> 4
00 -> 5
ill -> 6
wait -> 7
staanalystalert -> 8
jefferies -> 9
maintains -> 10
with -> 11
rating -> 12
hold -> 13
setting -> 14
target -> 15
price -> 16
usd -> 17
350 -> 18
our -> 19
own -> 20


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [12]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [13]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19552902003780573

In [14]:
#!cp ../../data/project_6_stocktwits/twits.json testfile.json
#!cp ../../data/project_6_stocktwits/test_twits.json test_twits.json


Finally let's convert our tokens into integer ids which we can pass to the network.

In [15]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [16]:
print (sentiments[0:10])
print (token_ids[:10])
print(type(token_ids))
print(len(token_ids))

twit_count=0
twit_length = 0 
for message in token_ids:
    twit_count+=1
    for word in message:
        twit_length +=1
        
print("counting twits in for loop to verify len -token-id :", twit_count)
print("average length per twit :", twit_length/twit_count)
    

[4, 4, 3, 3, 0, 3, 4, 0, 4, 0]
[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15, 16, 3, 17, 18, 5, 19, 20, 21, 2], [22, 23, 24, 25, 26, 27, 25, 28, 29, 26, 30, 31], [34, 35, 36, 37, 38, 39, 40, 41], [42, 43, 44, 45, 46, 47, 48, 49, 50], [51, 52, 53, 54, 55, 56, 16, 57, 58, 59], [60, 61, 53, 62, 63, 64, 65, 66], [35, 67, 68, 69, 70, 71, 72, 73, 74, 64, 75, 71, 76, 77, 78, 79, 80], [81, 2], [82, 83, 84, 3, 85, 45, 86, 82, 87, 88, 89]]
<class 'list'>
1037938
counting twits in for loop to verify len -token-id : 1037938
average length per twit : 10.613513523929175


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [17]:
# Below is my first attempt at the model, which gave an error 343
###########################################################
# class TextClassifier(nn.Module):
#     def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
#         """
#         Initialize the model by setting up the layers.
        
#         Parameters
#         ----------
#             vocab_size : The vocabulary size.
#             embed_size : The embedding layer size.
#             lstm_size : The LSTM layer size.
#             output_size : The output size.
#             lstm_layers : The number of LSTM layers.
#             dropout : The dropout probability.
#         """
        
#         super().__init__()
#         self.vocab_size = vocab_size
#         self.embed_size = embed_size
#         self.lstm_size = lstm_size
#         self.output_size = output_size
#         self.lstm_layers = lstm_layers
#         self.dropout = dropout
        
#         # TODO Implement

#         # Setup embedding layer
#         self.embedding = nn.Embedding(vocab_size, embed_size)
        
#         # Setup additional layers
#         self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout, batch_first=False)
#         self.dropout = nn.Dropout(dropout)
#         #Dense (linear fc) and softmax layer
#         self.fc = nn.Linear(lstm_size, output_size)
#         self.logsoftmax = nn.LogSoftmax(dim=1) # revoved dim-1 due to index error (343)


#     def init_hidden(self, batch_size):
#         """ 
#         Initializes hidden state
        
#         Parameters
#         ----------
#             batch_size : The size of batches.
        
#         Returns
#         -------
#             hidden_state
            
#         """
        
#         # TODO Implement 
        
#         # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
#         # initialized to zero, for hidden state and cell state of LSTM
        
#         weight = next(self.parameters()).data
#         hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(), 
#                   weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
#         return hidden


#     def forward(self, nn_input, hidden_state):
#         """
#         Perform a forward pass of our model on nn_input.
        
#         Parameters
#         ----------
#             nn_input : The batch of input to the NN.
#             hidden_state : The LSTM hidden state.

#         Returns
#         -------
#             logps: log softmax output
#             hidden_state: The new hidden state.

#         """
        
#         # TODO Implement
#         nn_input = nn_input.long()
#         embeds = self.embedding(nn_input)
#         lstm_out, hidden = self.lstm(embeds, hidden_state)
#         #return single output
#         lstm_out = lstm_out [-1,:,:] # getting the last time step output
#         #dropout and dense layer
#         out = self.dropout(lstm_out)
#         out = self.fc(out)
#         #softmax function
#         logps = self.logsoftmax(out)
#         print("LogSoftMax shape: ", logps)
        
#         # reshape to be batch_size last
#         batch_size = nn_input.size(1)
#         #logps = logps.view(-1, batch_size, self.output_size)
        
#         return logps, hidden

In [19]:
# this is my third model attempt using https://knowledge.udacity.com/questions/295065 directly as I could not figure out why my code was giving problems
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement
        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        # LSTM layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)
        
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        return (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.
        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.
        """
        
        # TODO Implement 
        # Set batch size
        batch_size = nn_input.size(1)
        # embeddings and lstm_out
        x = nn_input.long()
        embeds = self.embedding(x)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # softmax function
        log_ps = self.log_softmax(out)
        
        # reshape to be batch_size last
        log_ps = log_ps.view(-1, batch_size, self.output_size)
        log_ps = log_ps[-1, :, :] # get last batch of labels
        
        # return last softmax output and hidden state
        return log_ps, hidden_state

### View Model

In [20]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)


tensor([[-1.3480, -1.9564, -1.6402, -2.1598, -1.2393],
        [-1.3329, -1.9499, -1.6535, -2.1651, -1.2452],
        [-1.3511, -1.9516, -1.6393, -2.1470, -1.2446],
        [-1.3545, -1.9366, -1.6517, -2.1302, -1.2476]])


In [21]:
print(model)

TextClassifier(
  (embedding): Embedding(20661, 10)
  (lstm): LSTM(10, 6, num_layers=2, dropout=0.1)
  (dropout): Dropout(p=0.1)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [22]:
# matching my first attempt model. Please 
# def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
#     """ 
#     Build a dataloader.
#     """
#     if shuffle:
#         indices = list(range(len(messages)))
#         random.shuffle(indices)
#         messages = [messages[idx] for idx in indices]
#         labels = [labels[idx] for idx in indices]

#     total_sequences = len(messages)

#     for ii in range(0, total_sequences, batch_size):
#         batch_messages = messages[ii: ii+batch_size]
        
#         # First initialize a tensor of all zeros
#         batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
#         for batch_num, tokens in enumerate(batch_messages):
#             token_tensor = torch.tensor(tokens)
#             # Left pad!
#             start_idx = max(sequence_length - len(token_tensor), 0)
#             batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
#         label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
#         yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [23]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

In [24]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
# """   

# # TODO Implement 
# train_ratio=0.8
# threshold = int(len(token_ids)*train_ratio)

# train_features = token_ids[:threshold]
# valid_features = token_ids[threshold:]
# train_labels = sentiments[:threshold]
# valid_labels = sentiments[threshold:]

# TODO Implement 
split_frac = 0.7
split_idx = int(len(token_ids)*split_frac)

train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]

In [25]:
# text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
# model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
# hidden = model.init_hidden(64)
# logps, hidden = model.forward(text_batch, hidden)
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
print(text_batch)
print(text_batch.shape)
print(labels)
print(labels.shape)
print(len(vocab))
print(len(vocab)+1)
model = TextClassifier((len(vocab)+1), 200, 128, 5, dropout=0)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

tensor([[   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        ...,
        [   5,   20,   26,  ...,  179,  550,  554],
        [   6,   21,   30,  ...,  287,  551,    3],
        [   7,    2,   31,  ...,  232,  552,  101]])
torch.Size([20, 64])
tensor([ 4,  4,  3,  3,  0,  3,  4,  0,  4,  0,  4,  4,  4,  3,
         4,  4,  3,  3,  3,  0,  1,  3,  3,  0,  4,  4,  0,  3,
         1,  3,  3,  4,  3,  4,  3,  3,  3,  3,  3,  3,  1,  4,
         3,  4,  0,  3,  0,  2,  1,  4,  1,  3,  4,  1,  0,  2,
         2,  4,  2,  4,  2,  4,  2,  3])
torch.Size([64])
20661
20662


### Training
It's time to train the neural network!

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(20662, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [27]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
import numpy as np
epochs = 4
batch_size = 512
#learning_rate = 0.01
learning_rate = 0.005
clip= 5 # set gradient clipping.

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        
        # old code: model.zero_grad()
        # tip from Project Reiewer :x: You should use optimizer.zero_grad() to clean up the gradient instead of using model.zero_grad()
        # :x: You have to add the Accuracy metric when evaluating the validation dataset.
        
        #note that https://knowledge.udacity.com/questions/250563 listed model.zero_grad() as safer in general.
        #however as the model parameters are in the optimizer I have updated as per reviewer notes:
        
        optimizer.zero_grad()
        #forward pass
        output, hidden = model(text_batch, hidden)
        #backward - with grad norm clipping
        loss = criterion(output.squeeze(),labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(),clip)
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            val_losses = []
            val_accuracy = []
            #print("output at step ", steps)
            #print(output.shape)
            #print(output[0:5])
            #print("labels at step ", steps)
            #print(labels.shape)
            #print(labels[0:5])
            
            # TODO Implement: Print metrics
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size=batch_size):
                #creating new variables for the hidden state (to avoid backprogging through entire training history)
                inputs, labels = inputs.to(device), labels.to(device)
                val_h = model.init_hidden(labels.shape[0])
                for each in val_h:
                    each.to(device)
                    
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)
                val_losses.append(val_loss.item())
                #correct = (output.squeeze() == labels[-1]).float().sum()
                #total = len(output)
                #val_accuracy.append(correct/total)
                #print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss.data[0], correct/x.shape[0]))
                
                # referred to https://knowledge.udacity.com/questions/299391 for assistance in accuracy code.
                _, top_sentiment = torch.exp(output).topk(1)
                equality = top_sentiment == labels.view(*top_sentiment.shape)
                valid_accuracy = torch.mean(equality.type(torch.FloatTensor))
                val_accuracy.append(valid_accuracy.item())
            
            model.train()
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Accuracy: {:.6f}".format(np.mean(val_accuracy)))

Starting epoch 1
Epoch: 1/4... Step: 100... Loss: 0.916909... Val Loss: 0.952643 Val Accuracy: 0.624423
Epoch: 1/4... Step: 200... Loss: 0.905750... Val Loss: 0.877155 Val Accuracy: 0.660750
Epoch: 1/4... Step: 300... Loss: 0.800452... Val Loss: 0.827658 Val Accuracy: 0.678633
Epoch: 1/4... Step: 400... Loss: 0.768822... Val Loss: 0.800207 Val Accuracy: 0.693048
Epoch: 1/4... Step: 500... Loss: 0.794534... Val Loss: 0.784453 Val Accuracy: 0.697304
Epoch: 1/4... Step: 600... Loss: 0.741369... Val Loss: 0.777791 Val Accuracy: 0.698083
Epoch: 1/4... Step: 700... Loss: 0.702856... Val Loss: 0.764123 Val Accuracy: 0.701512
Epoch: 1/4... Step: 800... Loss: 0.735153... Val Loss: 0.764109 Val Accuracy: 0.704536
Epoch: 1/4... Step: 900... Loss: 0.695823... Val Loss: 0.759539 Val Accuracy: 0.707978
Epoch: 1/4... Step: 1000... Loss: 0.773032... Val Loss: 0.753889 Val Accuracy: 0.707297
Epoch: 1/4... Step: 1100... Loss: 0.697011... Val Loss: 0.755936 Val Accuracy: 0.710225
Epoch: 1/4... Step: 1200

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [28]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [token for token in tokens if token in vocab]
    # Convert words to ids
    tokens = [vocab[token] for token in tokens]
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1,1)))
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [29]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0004,  0.0113,  0.0099,  0.7795,  0.1990]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?

1. Prediction of the model:
The prediction distribution of the above text string has the highest score listed in the 4th bin, implying a positive sentiment, with 78 percent being alocated to that (second highest) bin. There is also almost 20 percent score for the highest sentiment.

2. Uncertainty of the prediction.
The model itself was run with 4 epocs giving a final accuracy of 72 percent and validation loss of 73 percent.
With more epochs and adjusted parameters, this can be improved, however the prediction for this simple text case for ticker GOOG performed very well as 97.8% of the binned sentiment is distributed in the upper 2 quantiles (and 78 % percent in the most common). 

As opposed to the validation stage which has a single label from -2 to 2 at source, the prediction can be spread with a probability distribution over the quintiles created for this project, as one can label the above text as either of the upper 2 quintiles; so there is systemic/epistemic uncertainty (see quoted paper below). If you consider that only 2.2% of the score being in the first 3 bins (negative and neutral), uncertainty is low (acceptable).



Quotation from paper:
http://mlg.eng.cam.ac.uk/yarin/thesis/thesis.pdf
Other situations that can lead to uncertainty include
• noisy data (our observed labels might be noisy, for example as a result of measurement imprecision, leading to aleatoric uncertainty),
• uncertainty in model parameters that best explain the observed data (a large
number of possible models might be able to explain a given dataset, in which case
we might be uncertain which model parameters to choose to predict with),
• and structure uncertainty (what model structure should we use? how do we specify
our model to extrapolate / interpolate well?).
The latter two uncertainties can be grouped under model uncertainty (also referred to
as epistemic uncertainty). Aleatoric uncertainty and epistemic uncertainty can then be
used to induce predictive uncertainty, the confidence we have in a prediction.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [30]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [31]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [32]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [33]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.1988,  0.0344,  0.0359,  0.0623,  0.6686]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.